In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#install dependencies
!pip install googledrivedownloader
!pip install --upgrade --no-cache-dir gdown
#!pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu111/torch_nightly.html -U

In [ ]:
# Name of the directory on Google Drive for saving training log, loss/mAP chart,
# and weights
SAVE_DIR = 'crowd'

# Path of the Google Drive mount point
DRIVE_DIR = '/content/drive/MyDrive'

# Full path to the save directory on Google Drive
DRIVE_SAVE_DIR = DRIVE_DIR + '/' + SAVE_DIR

# Input shape {width}*{height} of the yolov4 model to be trained
INPUT_SHAPE = '608x608'

In [ ]:
#clone repo
%cd /content
!git clone https://github.com/sirbudd/licenta.git
!mkdir /content/licenta/data/raw

In [ ]:
#download files
import gdown

%cd /content/licenta/data/raw

# #annotation_train.odgt
# url = 'https://drive.google.com/uc?id=1sNqB34uO-Go8tWYXQ7QCiQaKFXSlwXzk'
# output = 'annotation_train.odgt'
# gdown.download(url, output, quiet=False)

# #annotation_val.odg
# url = 'https://drive.google.com/uc?id=1p4-Z0RzP9kKA-8PKYdtPmOuhG1TwCdZf'
# output = 'annotation_val.odgt'
# gdown.download(url, output, quiet=False)

# #CrowdHuman_train01.zip
# url = 'https://drive.google.com/uc?id=1l9CvDD5Py5XTRQzcpgGYvf_QYX2dH-wG'
# output = 'CrowdHuman_train01.zip'
# gdown.download(url, output, quiet=False)

# #CrowdHuman_train02.zip
# url = 'https://drive.google.com/uc?id=1MGYDW1USUQgjyul49kSwOnrLahv5Q1HH'
# output = 'CrowdHuman_train02.zip'
# gdown.download(url, output, quiet=False)

# #CrowdHuman_train03.zip
# url = 'https://drive.google.com/uc?id=1xeHnpb9oaD8fsjAH9B_6sAL4J32wsV54'
# output = 'CrowdHuman_train03.zip'
# gdown.download(url, output, quiet=False)

# #CrowdHuman_val.zip
# url = 'https://drive.google.com/uc?id=1Cg9EX9CKUXXYVo7qYl44KgU2AU9wEv7j'
# output = 'CrowdHuman_val.zip'
# gdown.download(url, output, quiet=False)

!cp -r /content/drive/MyDrive/crowd-sets/CrowdHuman_train01.zip /content/licenta/data/raw
!cp -r /content/drive/MyDrive/crowd-sets/CrowdHuman_train02.zip /content/licenta/data/raw
!cp -r /content/drive/MyDrive/crowd-sets/CrowdHuman_train03.zip /content/licenta/data/raw
!cp -r /content/drive/MyDrive/crowd-sets/CrowdHuman_val.zip /content/licenta/data/raw
!cp -r /content/drive/MyDrive/crowd-sets/annotation_train.odgt /content/licenta/data/raw
!cp -r /content/drive/MyDrive/crowd-sets/annotation_val.odgt /content/licenta/data/raw

print("Done copying files!")

In [ ]:
#prepare data ---- extract and generate .jpg & .txt files
!chmod 755 /content/licenta/data/prepare_data.sh
%cd /content/licenta/data
#!dos2unix prepare_data.sh
!./prepare_data.sh {INPUT_SHAPE}

In [ ]:
ls /content/licenta/data/crowdhuman-416x416 | wc -l

In [ ]:
#download darknet from github --- ALEXEY VERSION
%cd /content/licenta
!rm -rf darknet
!git clone https://github.com/AlexeyAB/darknet.git

In [ ]:
%cd /content/licenta/darknet
!sed -i "1s/GPU=0/GPU=1/"               Makefile
!sed -i "2s/CUDNN=0/CUDNN=1/"           Makefile
# !sed -i "3s/CUDNN_HALF=0/CUDNN_HALF=1/" Makefile
!sed -i "4s/OPENCV=0/OPENCV=1/"         Makefile
!sed -i "5s/AVX=0/AVX=1/"               Makefile
!sed -i "7s/LIBSO=0/LIBSO=1/"           Makefile
# !sed -i "20s/compute_30,code=sm_30 \\\/compute_37,code=sm_37 -gencode arch=compute_60,code=[sm_60,compute_60] -gencode arch=compute_61,code=[sm_61,compute_61] -gencode arch=compute_75,code=[sm_75,compute_75]/" Makefile
# !sed -i "21s/^/#/"                      Makefile
# !sed -i "22s/^/#/"                      Makefile
# !sed -i "23s/^/#/"                      Makefile
# !sed -i "24s/^/#/"                      Makefile
# Let darknet test mAP more frequently during training 
!sed -i "300s/calc_map_for_each = 4/calc_map_for_each = 1/" src/detector.c

!make

In [ ]:
#Create folders and symlinks for saving training log and results.  --- clean old files
!mkdir -p "{DRIVE_SAVE_DIR}"
!rm -f "{DRIVE_SAVE_DIR}"/train.log
!touch "{DRIVE_SAVE_DIR}"/chart.png
!rm -f chart.png
!ln -sf "{DRIVE_SAVE_DIR}"/chart.png .
!mkdir -p "{DRIVE_SAVE_DIR}"/backup
!rm -rf backup
!ln -sf "{DRIVE_SAVE_DIR}"/backup .

In [ ]:
#Remaining setup pt training. Also download pre-trained weights
!chmod 755 /content/licenta/prepare_training.sh
%cd /content/licenta
!./prepare_training.sh {INPUT_SHAPE}

In [ ]:
#Start training
%cd /content/licenta/darknet
!./darknet detector train data/crowdhuman-{INPUT_SHAPE}.data cfg/yolov4-crowdhuman-{INPUT_SHAPE}.cfg yolov4.conv.137 -gpu 0 -map -dont_show 2>&1 | tee "{DRIVE_SAVE_DIR}"/train.log | grep -E "hours left|mean_average"
#!./darknet detector train data/crowdhuman-{INPUT_SHAPE}.data cfg/yolov4-crowdhuman-{INPUT_SHAPE}.cfg yolov4.conv.137 -map -dont_show | grep -E "hours left|mean_average"

In [ ]:
#TEST darknet
!./darknet detector test data/crowdhuman-{INPUT_SHAPE}.data cfg/yolov4-crowdhuman-{INPUT_SHAPE}.cfg backup/yolov4-crowdhuman-{INPUT_SHAPE}_best.weights /content/277961736_4251463678290414_6165787944070179067_n.jpg -gpu 0 -dont_show

from IPython.display import Image
Image('predictions.jpg')